In [1]:
import os

import numpy as np
import tensorflow as tf
from imutils import paths
from tensorflow.data import AUTOTUNE
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from pyimagesearch.helpers import benchmark

In [2]:
def load_images(imagePath):
    """
    read the image from disk, decode it, resize it,
    and scale the pixels intensities to the range [0, 1]

    :param imagePath:
    :return:
    """
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.resize(image, (96, 96)) / 255.0

    # grab the label and encode it
    label = tf.strings.split(imagePath, os.path.sep)[-2]
    oneHot = label == classNames
    encodedLabel = tf.argmax(oneHot)

    # return the image and the integer encoded label
    return image, encodedLabel

In [3]:
data_dir = 'fruits'

# initialize batch size and number of steps
BS = 64
NUM_STEPS = 1000

In [4]:
# grab the list of images in our dataset directory and grab all unique class names
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(data_dir))
classNames = np.array(sorted(os.listdir(data_dir)))

[INFO] loading image paths...


In [5]:
# create a standard image generator object
print("[INFO] creating a ImageDataGenerator object...")
imageGen = ImageDataGenerator(rescale=1.0 / 255)
dataGen = imageGen.flow_from_directory(
    data_dir,
    target_size=(96, 96),
    batch_size=BS,
    class_mode="categorical",
    color_mode="rgb"
)

[INFO] creating a ImageDataGenerator object...
Found 6688 images belonging to 7 classes.


In [6]:
# '''
# benchmark the image data generator and display the number of data points generated, 
# along with the time taken to perform the operation
# '''
# totalTime = benchmark(dataGen, NUM_STEPS)
# print(f"[INFO] ImageDataGenerator generated {BS * NUM_STEPS} images in {totalTime:.2f} seconds...")

In [7]:
# build the dataset and data input pipeline
print("[INFO] creating a tf.data input pipeline..")
dataset = tf.data.Dataset.from_tensor_slices(imagePaths)
dataset = dataset.shuffle(1024).map(load_images, num_parallel_calls=AUTOTUNE).cache().repeat().batch(BS).prefetch(AUTOTUNE)

[INFO] creating a tf.data input pipeline..


2022-06-20 14:59:25.245128: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:
'''
create a dataset iterator, benchmark the tf.data pipeline, 
and display the number of data points generated, along with the time taken
'''
datasetGen = iter(dataset)
totalTime = benchmark(datasetGen, NUM_STEPS)
print(f"[INFO] tf.data generated {BS * NUM_STEPS} images in {totalTime:.2f} seconds...")

[INFO] tf.data generated 64000 images in 22.67 seconds...


In [9]:
img_path = imagePaths[0]

In [12]:
x, y = load_images(img_path)

In [13]:
x

<tf.Tensor: shape=(96, 96, 3), dtype=float32, numpy=
array([[[0.00000000e+00, 3.92156886e-03, 1.96078438e-02],
        [0.00000000e+00, 0.00000000e+00, 7.84313772e-03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.38235301e-01, 1.64705887e-01, 1.75980389e-01],
        [1.21568628e-01, 3.72549035e-02, 1.47058824e-02],
        [1.85784310e-01, 1.60784319e-01, 1.03431374e-01]],

       [[0.00000000e+00, 0.00000000e+00, 1.56862754e-02],
        [0.00000000e+00, 4.90196107e-04, 8.33333377e-03],
        [0.00000000e+00, 3.92156886e-03, 0.00000000e+00],
        ...,
        [5.62745094e-01, 6.43137276e-01, 6.82843149e-01],
        [1.34803921e-01, 9.65686291e-02, 8.77451003e-02],
        [7.20588267e-02, 1.91176478e-02, 4.41176491e-03]],

       [[0.00000000e+00, 4.90196107e-04, 8.33333377e-03],
        [0.00000000e+00, 0.00000000e+00, 7.84313772e-03],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        ...,
        [1.95098042e-01, 2.9362744

In [14]:
y

<tf.Tensor: shape=(), dtype=int64, numpy=0>